<a href="https://colab.research.google.com/github/LeCorey/MastersCS/blob/main/Copy_of_Homework_2_Submitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

# Upload files
uploaded = files.upload()

# Read the files into pandas DataFrames
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

Saving train.csv to train (2).csv
Saving test.csv to test (2).csv
Saving data_description.txt to data_description (2).txt


In [ ]:
# Check for missing values in the training data
missing_values_train = train_data.isnull().sum()
missing_values_train = missing_values_train[missing_values_train > 0].sort_values(ascending=False)

# Checking the data types to identify categorical and numerical features
data_types = train_data.dtypes

# Display the missing values and data types
missing_values_train, data_types


(PoolQC          1453
 MiscFeature     1406
 Alley           1369
 Fence           1179
 FireplaceQu      690
 LotFrontage      259
 GarageType        81
 GarageYrBlt       81
 GarageFinish      81
 GarageQual        81
 GarageCond        81
 BsmtExposure      38
 BsmtFinType2      38
 BsmtFinType1      37
 BsmtCond          37
 BsmtQual          37
 MasVnrArea         8
 MasVnrType         8
 Electrical         1
 dtype: int64,
 Id                 int64
 MSSubClass         int64
 MSZoning          object
 LotFrontage      float64
 LotArea            int64
                   ...   
 MoSold             int64
 YrSold             int64
 SaleType          object
 SaleCondition     object
 SalePrice          int64
 Length: 81, dtype: object)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Dropping columns with excessive missing values
columns_to_drop = missing_values_train[missing_values_train > 200].index
train_data_reduced = train_data.drop(columns=columns_to_drop)

# Separating target variable and features
y = train_data_reduced['SalePrice']
X = train_data_reduced.drop(['SalePrice'], axis=1)

# Identifying numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Preprocessing for numerical data: imputation
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data: imputation + one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the training data
X_preprocessed = preprocessor.fit_transform(X)

# Shape of preprocessed data
X_preprocessed.shape


(1460, 270)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from math import sqrt

# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Columns that will be considered in the model
numerical_cols = [col for col in train_data.columns if train_data[col].dtype in ['int64', 'float64']]
categorical_cols = [col for col in train_data.columns if train_data[col].dtype == 'object']

# Adjusting the numerical columns to exclude 'SalePrice'
numerical_cols = [col for col in numerical_cols if col != 'SalePrice']

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = LinearRegression()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Separate target from predictors
y = train_data['SalePrice']
X = train_data.drop(['SalePrice'], axis=1)

# Split data into train and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Fitting the model
pipeline.fit(X_train, y_train)

# Preprocessing the test data and making predictions
test_preds = pipeline.predict(test_data)

# DataFrame the predictions
predictions_df = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': test_preds})

# Displaying the full predictions
print(predictions_df)


        Id      SalePrice
0     1461  111382.029669
1     1462  147255.875703
2     1463  184866.812172
3     1464  194577.062980
4     1465  195008.212096
...    ...            ...
1454  2915   80023.265015
1455  2916   80315.938063
1456  2917  181560.346136
1457  2918  114899.134275
1458  2919  214377.649999

[1459 rows x 2 columns]
